In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [20]:
pip install talos


In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
from scipy import stats
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import scale

# keres modules
from keras import regularizers
from keras.models import Sequential, load_model
from keras.layers import Dense, Activation, Dropout
from keras.layers import Flatten, Conv1D, MaxPooling1D
from keras.activations import relu, elu, linear, softmax
from keras.callbacks import EarlyStopping, Callback
from keras.wrappers.scikit_learn import KerasRegressor
from tensorflow.keras.optimizers import Adam # - Works

from keras.losses import mean_squared_error, categorical_crossentropy, logcosh
from keras.utils.np_utils import to_categorical



**Matrix Y contains the average grain yield, column 1: Grain yield for environment 1 and so on.**

**Matrix X contains marker genotypes.**


In [3]:
# load data as a pandas dataframe
X = pd.read_csv('/kaggle/input/genomicselection-data-weat/DATA/wheat.X', header=None, sep='\s+')
Y = pd.read_csv('/kaggle/input/genomicselection-data-weat/DATA/wheat.Y', header=None, sep='\s+')

In [4]:
X.head(10)
#print('#'*50)


In [5]:
print(X.shape)

In [6]:
print(Y.head(10))
print('#'*50)
print(Y.shape)

In [7]:
# data pattitioning into train and validation 
itrait=1
X_train, X_test, y_train, y_test = train_test_split(X, Y[itrait], test_size=0.2)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

In [8]:
# print basic statistics: max, min, mean, sd
print('      min max mean sd')
print('Train: ', y_train.min(), y_train.max(), y_train.mean(), np.sqrt(y_train.var()))
print('Test: ', y_test.min(), y_test.max(), y_test.mean(), np.sqrt(y_test.var()))


In [9]:
# basic histograms
plt.title('train / test data')
plt.hist(y_train, label='Train')
plt.hist(y_test, label='Test')
plt.legend(loc='best')
plt.show()

**Marker PCA, use whole x with different color for train and test**

In [12]:
X = np.concatenate((X_train, X_test))
pca = PCA(n_components=2)
p = pca.fit(X).fit_transform(X)
Ntrain=X_train.shape[0]
plt.title('PCA decomposition')
plt.scatter(p[0:Ntrain,0], p[0:Ntrain,1], label='Train')
plt.scatter(p[Ntrain:,0], p[Ntrain:,1], label='Test', color='orange')
plt.legend(loc='best')
plt.show()

**SNP preselection according to a simple GWAS**


In [13]:
pvals = []
for i in range(X_train.shape[1]):
    b, intercept, r_value, p_value, std_err = stats.linregress(X_train[i], y_train)
    pvals.append(-np.log10(p_value))
pvals = np.array(pvals)    

# plot GWAS
plt.ylabel('-log10 p-value')
plt.xlabel('SNP')
plt.plot(pvals, marker='o', color='red')
plt.show()

# select N_best most associated SNPs
# N_best = X_train.shape[1] # all SNPs
N_best = 100
snp_list = pvals.argsort()[-N_best:]

# select by min P_value
min_p_value = 2 
snp_list = np.nonzero(pvals>min_p_value)



**Standard penalized methods**
**lasso using scikit-learn**


In [14]:
# alpha is regularization parameter
lasso = linear_model.Lasso(alpha=0.01)
lasso.fit(X_train, y_train)
y_hat = lasso.predict(X_test)

# mean squared error
mse = mean_squared_error(y_test, y_hat)
print('\nMSE in prediction=', mse)

# correlation btw predicted and observed
corr = np.corrcoef(y_test,y_hat)[0,1]
print('\nCorr obs vs pred =', corr)



In [15]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')

# plot observed vs predicted targets
plt.title('Lasso: observed vs predicted Y')
plt.ylabel('Predicted')
plt.xlabel('Observed')
plt.scatter(y_test, y_hat, marker='o', cmap='viridis', alpha=0.3)
plt.show()

**Implements a standard fully connected neural network for quantitative targets**

In [16]:
# number of SNPs in data
nSNP = X_train.shape[1]
nSNP

**SoftPlus**

**is a smooth approximation to the ReLU function and can be used to constrain the output of a machine to always be positive**

In [17]:
# Instantiate
model = Sequential()

# add first layes
model.add(Dense(64, input_dim=nSNP))
model.add(Activation('relu'))
# add second layer
model.add(Dense(32))
model.add(Activation('softplus'))
# add third layer
model.add(Dense(32))
model.add(Activation('softplus'))
#last, output layer
model.add(Dense(1))

# Model Compiling (https://keras.io/models/sequential/) 
# compile(optimizer, loss=None, metrics=None, loss_weights=None, sample_weight_mode=None, weighted_metrics=None, target_tensors=None)
# Stochastic Gradient Descent (‘sgd’) as optimization algorithm
# Mean Squared Error as loss, ie, quantitative variable, regression
model.compile(loss='mean_squared_error', optimizer='sgd')

# list some properties
model.summary()

#tarining
## fit(x=None, y=None, batch_size=None, epochs=1, verbose=1, callbacks=None, validation_split=0.0, validation_data=None, shuffle=True, class_weight=None, sample_weight=None, initial_epoch=0, steps_per_epoch=None, validation_steps=None, validation_freq=1)
model.fit(X_train, y_train, epochs=50)

# cross-validation: get predicted target values
y_hat = model.predict(X_test, batch_size=128)

mse_prediction = model.evaluate(X_test, y_test, batch_size=128)
print('\MSE in prediction = ', mse_prediction)

# correlation btw predicted and observed
corr = np.corrcoef(y_test, y_hat[:,0])[0,1]
print('\Corr obs vs pred =', corr)

# plot observed vs predicted targets
plt.title('MLP: observed vs predicetd Y')
plt.ylabel('Predicted')
plt.xlabel('Observed')
plt.scatter(y_test, y_hat, marker='o')
plt.show()

**Controlling overfit: regularization, dropout and early stopping**

In [32]:
# deletes current model
del model

model = Sequential()

# Add l1 & l2 regularization in first layer
model.add(Dense(64, input_dim=nSNP,
                kernel_regularizer=regularizers.l2(0.01),
                activity_regularizer=regularizers.l1(0.01)))
model.add(Activation('relu'))
# Add second layer
model.add(Dense(32))
model.add(Activation('softplus'))
## Adding dropout to second layer
model.add(Dropout(0.2))
# Last, output layer
model.add(Dense(1))

# Model Compiling (https://keras.io/models/sequential/) 
model.compile(loss='mean_squared_error', optimizer='sgd')

# Split the train set into proper train & validation
X_train0, X_val, y_train0, y_val = train_test_split(X_train, y_train, test_size=0.1)
nEpochs=100

# Early stopping
early_stopper = EarlyStopping(monitor='val_loss', patience=10, min_delta=0.01)
model.fit(X_train0, y_train0, epochs=nEpochs, verbose=1, validation_data=(X_val, y_val), callbacks=[early_stopper])

# cross-validation
mse_prediction = model.evaluate(X_test, y_test, batch_size=128)
print('\nMSE in prediction =',mse_prediction)

# correlation btw predicted and observed
corr = np.corrcoef(y_test, y_hat[:,0])[0,1]
print('\Corr obs vs pred =', corr)

# plot observed vs predicted targets
plt.title('MLP: observed vs predicetd Y')
plt.ylabel('Predicted')
plt.xlabel('Observed')
plt.scatter(y_test, y_hat, marker='o')
plt.show()
## In this case neither l1 nor l2 regularization helps

In [27]:
# talos items (for hyperparameter search)
# import talos as ta
# import wrangle as wr
# from talos.metrics.keras_metrics import fmeasure_acc
# from talos.model.layers import hidden_layers
# from talos import live
# from talos.model import lr_normalizer, early_stopper, hidden_layers


In [22]:
# Defining pearson correlation as custom metric to model optimization in talos! 
# warning! you have to use acc in the metric name!

from keras import backend as K

def acc_pearson_r(y_true, y_pred):
    x = y_true
    y = y_pred
    mx = K.mean(x, axis=0)
    my = K.mean(y, axis=0)
    xm, ym = x - mx, y - my
    r_num = K.sum(xm * ym)
    x_square_sum = K.sum(xm * xm)
    y_square_sum = K.sum(ym * ym)
    r_den = K.sqrt(x_square_sum * y_square_sum)
    r = r_num / r_den
    return K.mean(r)

# Convolutional Neural Network

In [23]:
from keras.layers import Conv2D
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, LSTM


In [24]:
nSNP = X_train.shape[1]
nStride=3 # stride between convolutions
nFilter=32 # no. of convolutions

In [30]:
# Instantiate
model_cnn = Sequential()

# expand_dim is to match dimensions
X2_train = np.expand_dims(X_train,axis=2)
X2_test = np.expand_dims(X_test,axis=2)

# add conolutional layer
model_cnn.add(Conv1D(nFilter, kernel_size=3, strides=nStride, input_shape=(nSNP,1)))
# add pooling layer: takes maximun of two consecutive values
model_cnn.add(MaxPooling1D(pool_size=2))
# solutions above are linearized to accommdate a standard layers
model_cnn.add(Flatten())
model_cnn.add(Dense(64))
model_cnn.add(Activation('relu'))
model_cnn.add(Dense(64))
model_cnn.add(Activation('relu'))
model_cnn.add(Dense(32))
model_cnn.add(Activation('softplus'))
model_cnn.add(Dense(1))

# model compiling 
model_cnn.compile(loss='mean_squared_error', optimizer='sgd')

# list some properties
model_cnn.summary()

# tarining
model_cnn.fit(X2_train, y_train, epochs=100, verbose=0)





In [31]:
# cross-validation 
mse_prediction = model_cnn.evaluate(X2_test, y_test, batch_size=128)
print('\nMSE in prediction = ', mse_prediction)

# get predicted target values
y_hat = model_cnn.predict(X2_test, batch_size=128)

# correlation btw predicted and observed
corr = np.corrcoef(y_test,y_hat[:,0])[0,1]
print('\nCorr obs vs pred =',corr)


# plot observed vs predicted targets
plt.title('CNN: Observed vs Predicted Y')
plt.ylabel('Predicted')
plt.xlabel('Observed')
plt.scatter(y_test, y_hat, marker='o', alpha=0.3)
plt.show()